# Introduction to `linopy`

:::{note}
This material is in part adapted from the following resources:
- [Linopy Tutorial](https://linopy.readthedocs.io/en/latest/index.html)
- [PyPSA simple electricity market examples](https://pypsa.readthedocs.io/en/latest/examples/simple-electricity-market-examples.html)
:::



[Linopy](https://linopy.readthedocs.io/en/latest/index.html)  is an open-source framework for formulating, solving, and analyzing optimization problems with Python.

With Linopy, you can create an optimization model within Python that consists of decision variables, constraints, and an optimization objective. You can then solve these instances using a variety of commerci and open-source solvers (specialised software).

[Linopy](https://linopy.readthedocs.io/en/latest/index.html) supports a wide range of problem types, including:

- **Linear programming**
- Quadratic programmingng
- Nonlinear programming
- Mixed-integer linear programming
- Mixed-integer quadratic programming
- Mixed-integer nonlinear programming
- Stochastic programming
- Generalized disjunctive programming
- Differential algebraic equations
- Bilevel programmin- Mathematical programs with equilibrium constraints


:::{note}
Documentation for this package is available at https://linopy.readthedocs.io/en/latest/index.html.
:::.
:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pandas pyomo highspy
```
:::

### Initializing a `Model`

The Model class in Linopy is a fundamental part of the library. It serves as a container for all the relevant data associated with a linear optimization problem. This includes variables, constraints, and the objective function.

In [ ]:
import pandas as pd
import numpy as np

import xarray as xr

In [ ]:
from linopy import Model  # , Variable, Objective#, Constraint, Sense

In [ ]:
# Create a linopy Problem
m = Model()

This creates a new Model object, which you can then use to define your optimization problem.



### Adding variables

Variables in a linear optimization problem represent the decision variables. A variable can always be assigned with a lower and an upper bound. In our case, both `x` and `y` have a lower bound of zero (default is unbouded). In Linopy, you can add variables to a Model using the `add_variables` method:

In [ ]:
# Create Variables

x = m.add_variables(lower=0, name="x")
y = m.add_variables(lower=0, name="y");
# m.x = Variable(domain=(0, float('inf')))
# m.y = Variable()

`x` and `y` are linopy variables of the class `linopy.Variable`. Each of them contain all relevant information that define it. The `name` parameter is optional but can be useful for referencing the variables later.

In [ ]:
y

## Adding the Objective 

The objective function defines what you want to optimize. You can set the objective function of a Model in Linopy using the add_objective method. For our example that would be

In [ ]:
# Create Objective
m.add_objective(-4 * x - 3 * y)
# m.obj = Objective(4 * m.x + 3 * m.y, sense=Sense.MAXIMIZE)

Since both `x` and `y` are scalar variables (meaning they don't have any dimensions), their underlying data contain only one optimization variable each.  

### Adding Constraints

Constraints define the feasible region of the optimization problem. They consist of the left hand side (lhs) and the right hand side (rhs). The first constraint that we want to write down is 
$x <= 4$, which we write out exactly in the mathematical way.

We assign it by calling the function `m.add_constraints`.

In [ ]:
# Create Constraints

m.add_constraints(x <= 4)
m.add_constraints(2 * x + y <= 10)
m.add_constraints(x + y <= 10)

## Solving the Model

Once you've defined your Model with variables, constraints, and an objective function, you can solve it using the `solve` method:

In [ ]:
# Solve the Problem
m.solve()
# m.solve(solver="cbc")  # You can choose a different solver like 'glpk', 'gurobi', etc.

In [ ]:
x.solution.values

In [ ]:
y.solution.values

In [ ]:
# Print Solver Status
print("Solver Status:", m.status)

While we can read from the message above that our problem was solved successfully, we can also formally check by accessing the reported status in the Linopy "Model()" object.

In [ ]:
assert m.status == "ok"

## Electricity Market Examples

### Single bidding zone, single period

We want to minimise operational cost of an example electricity system representing South Africa subject to generator limits and meeting the load:

\begin{equation}
    \min_{g_s} \sum_s o_s g_s
  \end{equation}
  such that
  \begin{align}
    g_s &\leq G_s \\
    g_s &\geq 0 \\
    \sum_s g_s &= d
  \end{align}

We are given the following information on the South African electricity system:

Marginal costs in EUR/MWh

In [ ]:
marginal_costs = {
    "Wind": 0,
    "Coal": 30,
    "Gas": 60,
    "Oil": 80,
}

Power plant capacities in MW

In [ ]:
capacities = {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000}

Inelastic demand in MW

In [ ]:
load = 42000

In [ ]:
m = Model()

In [ ]:
fuel_type = pd.Index(capacities.keys(), name="fuel_type")
fuel_type

In [ ]:
capacities_series = pd.Series(capacities.values(), index=capacities.keys())
capacities_series

In [ ]:
g = m.add_variables(lower=0, upper=capacities_series, coords=[fuel_type], name="g")
g

In [ ]:
marginal_costs_new = {k: marginal_costs[k] for k in capacities.keys()}
marginal_costs_new

In [ ]:
marginal_costs_series = pd.Series(
    marginal_costs_new.values(), index=marginal_costs_new.keys()
)
marginal_costs_series

In [ ]:
g.sum() == load

It always helps to write out the constraints before adding them to the model. Since they look good, let’s assign them.

In [ ]:
m.add_constraints(g.sum() == load, name="total-generation-constraint")

In [ ]:
obj = (g * marginal_costs_series).sum()
m.add_objective(obj)

In [ ]:
m.solve()

In [ ]:
g.solution.fuel_type.values

In [ ]:
g.solution.values

In [ ]:
results = dict(zip(g.solution.fuel_type.values, g.solution.values))
results

### Two bidding zones with transmission

Let's add a spatial dimension, such that the optimisation problem is expanded to
\begin{equation}
  \min_{g_{i,s}, f_\ell} \sum_s o_{i,s} g_{i,s}
\end{equation}
such that
\begin{align}
  g_{i,s} &\leq G_{i,s} \\
  g_{i,s} &\geq 0 \\
  \sum_s g_{i,s} - \sum_\ell K_{i\ell} f_\ell &= d_i & \text{KCL} \\
  |f_\ell| &\leq F_\ell & \text{line limits}  \\
  \sum_\ell C_{\ell c} x_\ell f_\ell &= 0 & \text{KVL} 
\end{align}

In this example, we connect the previous South African electricity system with a hydro generation unit in Mozambique through a single transmission line. Note that because a single transmission line will not result in any cycles, we can neglect KVL in this case.

We are given the following data (all in MW):

In [ ]:
capacities = {
    "South Africa": {"Coal": 35000, "Wind": 3000, "Gas": 8000, "Oil": 2000, "Hydro": 0},
    "Mozambique": {"Coal": 0, "Wind": 0, "Gas": 0, "Oil": 0, "Hydro": 1200},
}

In [ ]:
transmission = 500

In [ ]:
loads = {"South Africa": 42000, "Mozambique": 650}

In [ ]:
marginal_costs = {
    "Wind": 0,
    "Coal": 30,
    "Gas": 60,
    "Oil": 80,
    "Hydro": 0,
}

In [ ]:
m = Model()

In [ ]:
capacities["South Africa"].keys()

In [ ]:
capacities["Mozambique"].keys()

In [ ]:
fuel_type = pd.Index(capacities["South Africa"].keys(), name="fuel_type")
fuel_type

In [ ]:
capacities_series_south_africa = pd.Series(
    capacities["South Africa"].values(), index=capacities["South Africa"].keys()
)
capacities_series_south_africa

In [ ]:
g_south_africa = m.add_variables(
    lower=0,
    upper=capacities_series_south_africa,
    coords=[fuel_type],
    name="g_south_africa",
)
g_south_africa

In [ ]:
capacities_series_mozambique = pd.Series(
    capacities["Mozambique"].values(), index=capacities["Mozambique"].keys()
)
capacities_series_mozambique

In [ ]:
g_mozambique = m.add_variables(
    lower=0, upper=capacities_series_mozambique, coords=[fuel_type], name="g_mozambique"
)
g_mozambique

In [ ]:
marginal_costs_new = {k: marginal_costs[k] for k in capacities["South Africa"].keys()}
marginal_costs_new

In [ ]:
marginal_costs_series = pd.Series(
    marginal_costs_new.values(), index=marginal_costs_new.keys()
)
marginal_costs_series

In [ ]:
line_flow = m.add_variables(
    lower=-1 * transmission, upper=transmission, name="line_flow"
)
line_flow

In [ ]:
g_south_africa.sum() - line_flow == loads["South Africa"]

In [ ]:
m.add_constraints(
    g_south_africa.sum() - line_flow == loads["South Africa"],
    name="total-generation-constraint-south-africa",
)

In [ ]:
g_mozambique.sum() + line_flow == loads["Mozambique"]

In [ ]:
m.add_constraints(
    g_mozambique.sum() + line_flow == loads["Mozambique"],
    name="total-generation-constraint-mozambique",
)

In [ ]:
obj = (g_south_africa * marginal_costs_series).sum() + (
    g_mozambique * marginal_costs_series
).sum()

m.add_objective(obj)

In [ ]:
m.solve()

In [ ]:
g_south_africa.solution.fuel_type.values

In [ ]:
g_south_africa.solution.values

In [ ]:
results_south_africa = dict(
    zip(g_south_africa.solution.fuel_type.values, g_south_africa.solution.values)
)
results_south_africa

In [ ]:
g_mozambique.solution.fuel_type.values

In [ ]:
g_mozambique.solution.values

In [ ]:
results_mozambique = dict(
    zip(g_mozambique.solution.fuel_type.values, g_mozambique.solution.values)
)
results_mozambique